***************************************************************************************
Jupyter Notebooks from the Metadata for Everyone project

Code:
* Dennis Donathan II (https://orcid.org/0000-0001-8042-0539)

Project team: 
* Juan Pablo Alperin (https://orcid.org/0000-0002-9344-7439)
* Dennis Donathan II (https://orcid.org/0000-0001-8042-0539)
* Mike Nason (https://orcid.org/0000-0001-5527-8489)
* Julie Shi (https://orcid.org/0000-0003-1242-1112)
* Marco Tullney (https://orcid.org/0000-0002-5111-2788)

Last updated: xxx
***************************************************************************************

# Data Cleaning
To clean this data set we'll start out by loading the dataset, checking for duplicates, and dropping columns that are not relevant to our analysis.

First, we'll load in our packages, set up our directories, and load in the dataset and take a look at it.

In [1]:
import modin.pandas as pd
import numpy as np

from pathlib import Path

#Set up directories
data_dir = Path('../data')
input_dir = data_dir / 'input'
output_dir = data_dir / 'output'

df = pd.read_csv(input_dir / '01_raw_data.csv', low_memory=False)

2024-04-27 17:44:20,788	INFO worker.py:1749 -- Started a local Ray instance.
(raylet) Spilled 3173 MiB, 25 objects, write throughput 1024 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


## Duplicate Records
Looking at the shape of the dataset against the number of unique DOIs will let us know just how many duplicate records we have.


In [2]:
df.shape

(531800, 47)

In [3]:
len(set(df['DOI']))

530043

In [4]:
#Dropping duplicate records
df.drop_duplicates(subset=['DOI'], keep='first', inplace=True)
df.shape

(raylet) Spilled 4365 MiB, 49 objects, write throughput 1144 MiB/s.


(530043, 47)

## Editors
There are very few records that have a value in the *editor* column. Some of our prior work indicates that this can be a sign of a work that has been mislabeled as a 'journal article'. So we'll explore some of the records with a value in the editor column in order to verify that.

We'll set up a dataframe of just those records that have data in the *editor* column.

Next, we'll search the titles of these records for a few keywords.

In [5]:
editorial = df.loc[df.title.str.contains(r'editorial|errata|contents|conference|proceedings|masthead|symposium|abstract|Book Review|preface|title page', 
                                         regex=True, case=False, na=False)]
editorial

,indexed,reference-count,publisher,license,content-domain,short-container-title,abstract,DOI,type,created,...,alternative-id,subject,article-number,issue,journal-issue,subtitle,original-title,archive,editor,published-other
24,"{'date-parts': [[2022, 4, 5]], 'date-time': '2...",0,Sri Lanka Journals Online (JOL),NaN,"{'domain': [], 'crossmark-restriction': False}",['J. Geol. Soc. Sri Lanka'],NaN,10.4038/jgssl.v21i1.33,journal-article,"{'date-parts': [[2020, 7, 13]], 'date-time': '...",...,NaN,"['Electrical and Electronic Engineering', 'Ato...",NaN,1,"{'issue': '1', 'published-online': {'date-part...",NaN,NaN,NaN,NaN,NaN
32,"{'date-parts': [[2023, 10, 25]], 'date-time': ...",1,Wiley,"[{'start': {'date-parts': [[2010, 8, 19]], 'da...","{'domain': [], 'crossmark-restriction': False}",['ChemInform'],<jats:title>Abstract</jats:title><jats:p>ChemI...,10.1002/chin.199433129,journal-article,"{'date-parts': [[2010, 8, 31]], 'date-time': '...",...,['10.1002/chin.199433129'],"['General Earth and Planetary Sciences', 'Gene...",NaN,33,"{'issue': '33', 'published-print': {'date-part...",NaN,NaN,['Portico'],NaN,NaN
35,"{'date-parts': [[2022, 4, 3]], 'date-time': '2...",0,Institute of Electrical and Electronics Engine...,"[{'start': {'date-parts': [[2016, 9, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['IEEE Trans. Smart Grid'],NaN,10.1109/tsg.2016.2595438,journal-article,"{'date-parts': [[2016, 8, 19]], 'date-time': '...",...,NaN,['General Computer Science'],NaN,5,{'issue': '5'},NaN,NaN,NaN,NaN,NaN
45,"{'date-parts': [[2022, 3, 29]], 'date-time': '...",0,Massachusetts Medical Society,NaN,"{'domain': [], 'crossmark-restriction': False}",['The Boston Medical and Surgical Journal'],NaN,10.1056/nejm192608121950728,journal-article,"{'date-parts': [[2011, 11, 11]], 'date-time': ...",...,['10.1056/NEJM192608121950728'],"['General Medicine', 'Management Science and O...",NaN,7,"{'issue': '7', 'published-print': {'date-parts...",NaN,NaN,NaN,NaN,NaN
117,"{'date-parts': [[2022, 4, 2]], 'date-time': '2...",0,Test accounts,NaN,"{'domain': [], 'crossmark-restriction': False}",['Bulletin'],NaN,10.1306/ad461ca0-16f7-11d7-8645000102c1865d,journal-article,"{'date-parts': [[2002, 12, 31]], 'date-time': ...",...,['AD461CA0-16F7-11D7-8645000102C1865D'],['Earth and Planetary Sciences (miscellaneous)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531607,"{'date-parts': [[2023, 10, 24]], 'date-time': ...",0,Wiley,"[{'start': {'date-parts': [[2007, 5, 31]], 'da...","{'domain': [], 'crossmark-restriction': False}",['Journal of Art &amp; Design Education'],<jats:p>Book reviewed in this article:</jats:p...,10.1111/j.1476-8070.1992.tb00314.x,journal-article,"{'date-parts': [[2007, 5, 31]], 'date-time': '...",...,['10.1111/j.1476-8070.1992.tb00314.x'],['General Medicine'],NaN,2,"{'issue': '2', 'published-print': {'date-parts...",NaN,NaN,['Portico'],NaN,NaN
531642,"{'date-parts': [[2022, 3, 28]], 'date-time': '...",0,Elsevier BV,"[{'start': {'date-parts': [[2020, 10, 1]], 'da...","{'domain': [], 'crossmark-restriction': False}",['Omega'],NaN,10.1016/s0305-0483(20)30645-9,journal-article,"{'date-parts': [[2020, 6, 16]], 'date-time': '...",...,['S0305048320306459'],"['Information Systems and Management', 'Manage...",102291,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531673,"{'date-parts': [[2022, 3, 31]], 'date-time': '...",0,SAGE Publications,"[{'start': {'date-parts': [[1907, 10, 1]], 'da...","{'domain': [], 'crossmark-restriction': False}",['Journal of Education'],NaN,10.1177/002205740706601423,journal-article,"{'date-parts': [[2018, 8, 21]], 'date-time': '...",...,['10.1177/002205740706601423'],['Education'],NaN,14,"{'issue': '14', 'published-print': {'date-part...",NaN,NaN,NaN,NaN,NaN
531680,"{'date-parts': [[2023, 10, 10]], 'date-time': ...",0,Wiley,"[{'start': {'date-parts': [[2012, 12, 18]], 'd...","{'domain': ['onlinelibrary.wiley.com'], 'cross...",['Magnetic Resonance in Med'],NaN,10.1002

We've found some editorials, Mastheads, conference proceedings, and abstracts. We'll go ahead and drop them from our dataset.

In [6]:
df.drop(editorial.index, inplace=True)

In [7]:
df.shape

(512875, 47)

## Conferences
Looking back at **editorial** we see that there are a couple 'Conferences' and 'Proceedings' in the *container-title* column. Let's take a look at just how many records remain in our dataset are from these journals/containers.

Additionally, we see a few records from the journal *ChemInform*, a journal that publishes chemistry abstracts, we'll check to see if any of those records remain as well.

We'll use a keyword search in the *container-title* column to find these records.

In [8]:
conferences = df.loc[(df['container-title'].str.contains(r'conference|ChemInform|news|CrossRef Listing of Deleted DOIs', regex=True, case=False)) | (df.publisher == 'EDP Sciences')]
conferences

(raylet) Spilled 8605 MiB, 120 objects, write throughput 1353 MiB/s.


,indexed,reference-count,publisher,license,content-domain,short-container-title,abstract,DOI,type,created,...,alternative-id,subject,article-number,issue,journal-issue,subtitle,original-title,archive,editor,published-other
47,"{'date-parts': [[2022, 4, 6]], 'date-time': '2...",0,Japan Society of Mechanical Engineers,NaN,"{'domain': [], 'crossmark-restriction': False}",NaN,NaN,10.1299/jsmemp.2001.9.71,journal-article,"{'date-parts': [[2017, 6, 25]], 'date-time': '...",...,NaN,NaN,NaN,0,"{'issue': '0', 'published-print': {'date-parts...",NaN,['212 ブレーキタイミングが摩擦圧接継手の機械的性質に及ぼす影響(OS 溶接・接合)'],NaN,NaN,NaN
61,"{'date-parts': [[2023, 10, 23]], 'date-time': ...",0,Wiley,"[{'start': {'date-parts': [[2011, 8, 9]], 'dat...",{'domain': ['anthrosource.onlinelibrary.wiley....,['Anthropology News'],NaN,10.1111/j.1556-3502.1990.tb01544.x,journal-article,"{'date-parts': [[2011, 8, 9]], 'date-time': '2...",...,['10.1111/j.1556-3502.1990.tb01544.x'],['General Medicine'],NaN,1,"{'issue': '1', 'published-print': {'date-parts...",NaN,NaN,['Portico'],NaN,NaN
181,"{'date-parts': [[2022, 4, 4]], 'date-time': '2...",0,Wiley,"[{'start': {'date-parts': [[2015, 9, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['Science News'],NaN,10.2307/3977842,journal-article,"{'date-parts': [[2007, 11, 27]], 'date-time': ...",...,NaN,['General Engineering'],NaN,5,"{'issue': '5', 'published-print': {'date-parts...",NaN,NaN,NaN,NaN,NaN
267,"{'date-parts': [[2022, 4, 4]], 'date-time': '2...",0,Rezekne Academy of Technologies,NaN,"{'domain': [], 'crossmark-restriction': False}",['AMCD'],"<jats:p>Every person, group or organization ha...",10.17770/amcd2016.2198,journal-article,"{'date-parts': [[2016, 11, 29]], 'date-time': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338,"{'date-parts': [[2022, 4, 3]], 'date-time': '2...",6,IOP Publishing,"[{'start': {'date-parts': [[2019, 10, 1]], 'da...","{'domain': ['iopscience.iop.org'], 'crossmark-...",['IOP Conf. Ser.: Earth Environ. Sci.'],<jats:title>Abstract</jats:title>\n ...,10.1088/1755-1315/343/1/012209,journal-article,"{'date-parts': [[2019, 11, 6]], 'date-time': '...",...,NaN,['General Medicine'],NaN,1,"{'issue': '1', 'published-print': {'date-parts...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531648,"{'date-parts': [[2022, 3, 31]], 'date-time': '...",11,EDP Sciences,"[{'start': {'date-parts': [[2018, 9, 14]], 'da...","{'domain': [], 'crossmark-restriction': False}",['E3S Web Conf.'],<jats:p>The existing building energy-saving re...,10.1051/e3sconf/20185301013,journal-article,"{'date-parts': [[2018, 9, 14]], 'date-time': '...",...,['e3sconf_icaeer2018_01013'],"['Pulmonary and Respiratory Medicine', 'Pediat...",NaN,NaN,NaN,NaN,NaN,NaN,"[{'given': 'C.-H.', 'family': 'Weng', 'sequenc...",NaN
531682,"{'date-parts': [[2022, 3, 30]], 'date-time': '...",8,IOP Publishing,"[{'start': {'date-parts': [[2019, 3, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['J. Phys.: Conf. Ser.'],NaN,10.1088/1742-6596/1176/4/042092,journal-article,"{'date-parts': [[2019, 3, 20]], 'date-time': '...",...,NaN,['General Physics and Astronomy'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531723,"{'date-parts': [[2022, 3, 29]], 'date-time': '...",0,EDP Sciences,NaN,"{'domain': [], 'crossmark-restriction': False}",['Environ. Biosafety Res.'],NaN,10.1051/ebr:2007008,journal-article,"{'date-parts': [[2007, 7, 19]], 'date-time': '...",...,['ebr0707'],"['Safety Research', 'General Agricultural and ...",NaN,4,{'issue': '4'},NaN,NaN,NaN,NaN,NaN
531777,"{'date-parts': [[2023, 10, 19]], 'date-time': ...",1,Wiley,"[{'start': {'date-parts': [[2003, 6, 26]], 'da...","{'domain': [], 'crossmark-restriction': False}",['ChemInform'],<jats:title>Abstract</jats:title><jats:p>For A...,10.1002/chin.200325028,journal-article,"{'date-parts': [[2005, 6, 29]], 'date-time': '...",...,['10.1002/chin.200325028'],"['General Earth and Planetary Sciences', 'Gene...",NaN,25,"{'issue': '25', 'publishe

In [9]:
df.drop(conferences.index, inplace=True)
df.shape

(504706, 47)

## Cleaning Dates
Here we are going to re-format some of the datetime columns into a more easily parsed format. *Created, deposited* and *published*. Not all records have month and day values for the *published* field, so we'll only take the year from those. For *created* and *deposited* we will have a YYYY-MM-DD format.

We've chosen these dates because they reflect certain information that we'll use later on. *Created* is the date when the item was first inserted into the Crossref database. *Deposited* reflects the last time the record was entered by the publisher (potentially with changes to the record but not necessarily the case). *Published* reflects when the item itself was actually published.

We'll use a regular expression to extract the dates from each of the records in each of those three columns, then we'll convert them to datetime dtypes.

In [10]:
date_columns = ['created', 'deposited']

for col in date_columns:
    df[col] = df[col].str.extract(r"\'(\d{4}\S\d{2}\S\d{2})")
    df[col] = pd.to_datetime(df[col], format="%Y-%m-%d")
df['published'] = df['published'].str.extract(r"(\d{4})")
df

,indexed,reference-count,publisher,license,content-domain,short-container-title,abstract,DOI,type,created,...,alternative-id,subject,article-number,issue,journal-issue,subtitle,original-title,archive,editor,published-other
0,"{'date-parts': [[2022, 4, 5]], 'date-time': '2...",0,F1000 Research Ltd,"[{'start': {'date-parts': [[2017, 10, 6]], 'da...","{'domain': ['mededpublish.org'], 'crossmark-re...",['MedEdPublish'],<ns4:p>This article was migrated. The article ...,10.15694/mep.2017.000176,journal-article,2017-10-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'date-parts': [[2023, 1, 14]], 'date-time': '...",55,Elsevier BV,"[{'start': {'date-parts': [[2021, 8, 1]], 'dat...","{'domain': ['elsevier.com', 'sciencedirect.com...",['Cretaceous Research'],NaN,10.1016/j.cretres.2021.104801,journal-article,2021-03-23,...,['S0195667121000483'],['Paleontology'],104801,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'date-parts': [[2022, 3, 30]], 'date-time': '...",9,SAGE Publications,"[{'start': {'date-parts': [[1997, 6, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['Educational and Psychological Measurement'],<jats:p> The purpose of this study was to inve...,10.1177/0013164497057003006,journal-article,2007-03-11,...,['10.1177/0013164497057003006'],"['Applied Mathematics', 'Applied Psychology', ...",NaN,3,"{'issue': '3', 'published-print': {'date-parts...",NaN,NaN,NaN,NaN,NaN
3,"{'date-parts': [[2022, 12, 15]], 'date-time': ...",0,Editions Techniques de l Ingenieur,NaN,"{'domain': [], 'crossmark-restriction': False}",NaN,NaN,10.51257/a-v1-n4850,journal-article,2022-09-29,...,NaN,"['Sensory Systems', 'Physiology', 'Museology',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'date-parts': [[2022, 3, 31]], 'date-time': '...",0,Ovid Technologies (Wolters Kluwer Health),NaN,"{'domain': [], 'crossmark-restriction': False}",['Anesthesiology'],NaN,10.1097/00000542-200209002-00408,journal-article,2014-10-18,...,NaN,['Anesthesiology and Pain Medicine'],NaN,Sup 2,{'issue': 'Sup 2'},['[2002][A-408]'],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531795,"{'date-parts': [[2023, 9, 16]], 'date-time': '...",44,Elsevier BV,"[{'start': {'date-parts': [[2019, 11, 1]], 'da...","{'domain': ['elsevier.com', 'sciencedirect.com...",['Mechanical Systems and Signal Processing'],NaN,10.1016/j.ymssp.2019.106260,journal-article,2019-08-08,...,['S0888327019304753'],"['Computer Science Applications', 'Mechanical ...",106260,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531796,"{'date-parts': [[2022, 6, 15]], 'date-time': '...",49,Universidad Pedagogica y Tecnologica de Colombia,NaN,"{'domain': [], 'crossmark-restriction': False}",['Cuad. linguist. hisp.'],<jats:p>This paper reports the findings of a q...,10.19053/0121053x.n38.2021.11205,journal-article,2021-11-03,...,NaN,['General Medicine'],NaN,38,"{'issue': '38', 'published-online': {'date-par...",NaN,NaN,NaN,NaN,NaN
531797,"{'date-parts': [[2023, 11, 8]], 'date-time': '...",3,Ovid Technologies (Wolters Kluwer Health),NaN,"{'domain': [], 'crossmark-restriction': False}",NaN,NaN,10.1097/aco.0b013e3280895aa3,journal-article,2007-03-07,...,NaN,['Anesthesiology and Pain Medicine'],NaN,2,"{'issue': '2', 'published-print': {'date-parts...",NaN,NaN,NaN,NaN,NaN
531798,"{'date-parts': [[2023, 10, 4]], 'date-time': '...",0,Programa de Pós-Graduação em Filosofia da Univ...,"[{'start': {'date-parts': [[2023, 9, 4]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['Revista TRÁGICA'],<jats:p>Tradução de Strategia del ragno (de Be...,10.59488/tragica.v16i2.60772,journal-article,2023-09-05,...,NaN,NaN,NaN,2,"{'issue': '2', 'published-online': {'date-part...",NaN,NaN,NaN,NaN,NaN


# String slicing
Now that the dates are converted, one of the last problems to address are some of the excess character in the *title, short-container-title,* and *container-title* fields.

In [11]:
cols = ['title', 'short-container-title', 'container-title']
for col in cols:
    df[col] = df[col].str.slice(start=2, stop=-2)

In [12]:
df['title'][0]

'A Modified Team Based Learning Approach to Enhance Resident Education in Outpatient Internal Medicine'

## Cleaning XML tags
We'll be looking at the abstract column, so it will benefit us to clean out the tags and only have te relevant text for each record. We'll write a quick function to do that.

In [13]:
df['abstract'][16]

"<jats:p>Human life increasingly modern along with the times. However, modernity leads them to become unsafe and uncomfortable living life. Peace of life can only be achieved through the right attitude towards the treasures and sparkles of the other world. This attitude is known as qana'ah, which means feeling sufficient and satisfied for what he has. This study aims to describe the study of qana'ah according to Hamka and to determine its implications in mental health. It\xa0is a literature study that using descriptive qualitative methods. The results of this study indicate that qana'ah according to Hamka is an attitude that accepts what it is, but still must make the effort. Qana'ah requires the sincerity of the heart in accepting what is owned by accompanied by maximum effort. If someone practices the concept of qana'ah Hamka, then a healthy mental will be formed. The application of qana'ah in daily life can make a person always optimistic, never give up and not greedy in everything.

In [14]:
df.loc[16, :]

indexed                   {'date-parts': [[2022, 3, 31]], 'date-time': '...
reference-count                                                           0
publisher                           Omah Jurnal Sunan Giri, INSURI Ponorogo
license                   [{'start': {'date-parts': [[2020, 10, 24]], 'd...
content-domain               {'domain': [], 'crossmark-restriction': False}
short-container-title                                              muharrik
abstract                  <jats:p>Human life increasingly modern along w...
DOI                                             10.37680/muharrik.v3i02.465
type                                                        journal-article
created                                                 2020-10-24 00:00:00
page                                                                227-243
update-policy                                                           NaN
source                                                             Crossref
is-reference

In [15]:
#import beautiful soup
from bs4 import BeautifulSoup as bs
def clean_abstracts(abstract):
    try:
        soup = bs(abstract, features='lxml')
        stripped_strings = soup.get_text()
        return stripped_strings
    except:
        return None
stripped_abstracts = df.abstract.map(lambda x: clean_abstracts(x))
df['abstract'] = stripped_abstracts

In [16]:
df['abstract'][16]

In [17]:
# make sure all missing values are correctly represented
import numpy as np
df = df.fillna(value=np.nan)

(raylet) Spilled 17064 MiB, 392 objects, write throughput 1451 MiB/s.


Looks great! Now we'll save our cleaned dataset.

In [18]:
df.to_csv(input_dir / '02_cleaned_data.csv', index=False)